# Step 1: Set Up Environment


In [1]:
pip install tensorflow matplotlib numpy


# Step 2: Load and Preprocess the MNIST Dataset


In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# Load the MNIST dataset
(x_train, _), (x_test, _) = tf.keras.datasets.mnist.load_data()

# Normalize the images to values between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Reshape the data to include a channel dimension (for grayscale images)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Step 3: Define the β-VAE Architecture


**Encoder**

In [3]:
def build_encoder(latent_dim):
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(28, 28, 1)),
        layers.Conv2D(32, 3, activation='relu', strides=2, padding='same'),
        layers.Conv2D(64, 3, activation='relu', strides=2, padding='same'),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(latent_dim * 2)  # Latent space mean and log variance
    ])
    return model


**Decoder**


In [4]:
def build_decoder(latent_dim):
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(latent_dim,)),
        layers.Dense(7 * 7 * 64, activation='relu'),
        layers.Reshape((7, 7, 64)),
        layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same'),
        layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same'),
        layers.Conv2DTranspose(1, 3, activation='sigmoid', padding='same')  # Output layer
    ])
    return model


# Step 4: Define the Loss Function with β Regularization


In [5]:
def compute_loss(x, x_reconstructed, z_mean, z_log_var, beta=1.0):
    # Reconstruction loss (binary cross-entropy)
    reconstruction_loss = tf.reduce_mean(tf.reduce_sum(
        tf.keras.losses.binary_crossentropy(x, x_reconstructed), axis=(1, 2)))

    # KL Divergence loss (regularization)
    kl_loss = -0.5 * tf.reduce_mean(tf.reduce_sum(
        1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1))

    # Total loss
    total_loss = reconstruction_loss + beta * kl_loss
    return total_loss


In [8]:
latent_dim = 2  # Latent space dimensionality
beta = 4  # Beta value to control disentangling

encoder = build_encoder(latent_dim)
decoder = build_decoder(latent_dim)

# Define the model
inputs = tf.keras.Input(shape=(28, 28, 1))
encoder_output = encoder(inputs)

# Use a Lambda layer for the split operation
z_mean, z_log_var = layers.Lambda(lambda x: tf.split(x, num_or_size_splits=2, axis=1))(encoder_output)

z = z_mean + tf.exp(0.5 * z_log_var) * tf.random.normal(shape=tf.shape(z_mean))  # Reparameterization trick
x_reconstructed = decoder(z)

# Compile the model
vae = tf.keras.Model(inputs, x_reconstructed)
vae.compile(optimizer=tf.keras.optimizers.Adam())

# Training step function
@tf.function
def train_step(x):
    with tf.GradientTape() as tape:
        x_reconstructed = vae(x)
        z_mean, z_log_var = tf.split(encoder(x), num_or_size_splits=2, axis=1)
        loss = compute_loss(x, x_reconstructed, z_mean, z_log_var, beta)
    grads = tape.gradient(loss, vae.trainable_variables)
    vae.optimizer.apply_gradients(zip(grads, vae.trainable_variables))
    return loss

# Training loop
epochs = 10
batch_size = 64
for epoch in range(epochs):
    for i in range(0, len(x_train), batch_size):
        batch = x_train[i:i+batch_size]
        loss = train_step(batch)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.numpy()}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
def generate_images(decoder, latent_dim, n_samples=10):
    z = np.random.randn(n_samples, latent_dim)
    generated_images = decoder.predict(z)
    generated_images = np.squeeze(generated_images, axis=-1)  # Remove the channel dimension
    return generated_images

# Generate new images
generated_images = generate_images(decoder, latent_dim)

# Display generated images
plt.figure(figsize=(10, 10))
for i in range(10):
    plt.subplot(1, 10, i+1)
    plt.imshow(generated_images[i], cmap='gray')
    plt.axis('off')
plt.show()


In [ ]:
# Manipulating the latent variables
def manipulate_latent(decoder, latent_dim, z_latent, factor=1.5):
    z_latent_modified = z_latent * factor
    return decoder.predict(z_latent_modified)

# Create a sample latent vector
sample_latent = np.random.randn(1, latent_dim)

# Modify the latent vector and generate a new image
modified_image = manipulate_latent(decoder, latent_dim, sample_latent, factor=1.5)

# Display the modified image
plt.imshow(modified_image.squeeze(), cmap='gray')
plt.axis('off')
plt.show()
